In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.19.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install sentence_transformers
#!pip install konlpy

!pip uninstall pandas
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/dist-packages/')

1.21.6


In [ ]:
print(numpy.version.version)

NotImplementedError: ignored

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")
"김형호 영화시장 분석가는 '1987'의 네이버 영화 정보 네티즌 10점 평에서 언급된 단어들을 지난해 12월 27일부터 올해 1월 10일까지 통계 프로그램 R과 KoNLP 패키지로 텍스트마이닝하여 분석했다."
# Apply a list of words that must be non-spacing
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')

In [ ]:
import numpy as np
import itertools
import pandas as pd



from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI/KeyBERT/CAI_data_002.csv')

In [ ]:
doc = df['sentence']

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [ ]:
tokenized_docs = []
no_token = []
tokenized_nounses = []
no_noun = []


for i in range(len(doc)) :
  okt = Okt()

  tokenized_doc = okt.pos(doc[i])
  if len(tokenized_doc) >= 2 : 
    tokenized_docs.append(tokenized_doc)

# tokenized_nouns = ' '.join([word[0] for word in tokenized_docs if word[1] == 'Noun'])
# if len(tokenized_doc) >= 2 :
#   tokenized_nounses.append(tokenized_nouns)

In [ ]:
tokenized_nounses = []
for word_list in tokenized_docs:
  preprocess_list = []
  for word in word_list:
    preprocess_list.append(word[0])
  tokenized_nounses.append([' '.join(preprocess_list)])

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
원본 : 부팅 중 오류진단 정상포트리셋 접수일 시
문장의 키워드 : ['부팅 오류', '진단 정상']
trigram 개수 : 11
trigram 다섯개만 출력 : ['리셋 접수' '부팅 오류' '부팅 오류 진단' '오류 진단' '오류 진단 정상']
------------------------------------------------------------------------------------------------------
원본 : 화질이상 시청은 되지만 줄이 여러 개 생긴다고 하심 연 후방 추티업 실패
문장의 키워드 : ['줄이 여러', '생긴다고 후방 추티업']
trigram 개수 : 17
trigram 다섯개만 출력 : ['되지만 줄이' '되지만 줄이 여러' '생긴다고 후방' '생긴다고 후방 추티업' '시청 되지만']
------------------------------------------------------------------------------------------------------
원본 : 차 차례 리콜 연결 안 됨 접수 보류 전송차 고객 요청사항리셋 접수 일부 채널 수신 불재전 송재부팅 제시회선 단말특이사항 진단 결과 기타 추가 연락처
문장의 키워드 : ['기타 추가 연락처', '전송 고객 요청']
trigram 개수 : 51
trigram 다섯개만 출력 : ['결과 기타' '결과 기타 추가' '고객 요청' '고객 요청 사항' '기타 추가']
------------------------------------------------------------------------------------------------------
원본 : 고객 요청사항리셋 접수 건수신불연 후방 독촉 취소 건이나 근처 방문 시 우선 방문 요망회선 단말특이사항 진단 결과 기타 추가 연락처
문장의 키워드 : ['기타 추가 연락처', '리셋 접수 불연']
trigram 개수 : 45
trigram

In [ ]:
for i in range(len(tokenized_nounses)) :
  try:
    n_gram_range = (2, 3)
    count = CountVectorizer(ngram_range=n_gram_range).fit(tokenized_nounses[i])
    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    top_n = 2
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    # keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=2, nr_candidates=10)
    print('원본 :', doc[i]) 
    print('문장의 키워드 :' , keywords,)
        

    print('trigram 개수 :',len(candidates))
    print('trigram 다섯개만 출력 :',candidates[:5])
    print('------------------------------------------------------------------------------------------------------') 
  except ValueError as e:
    print('불용어만 있습니다', e)
    print('------------------------------------------------------------------------------------------------------') 
  except TypeError as c:
    print('Nonetype이 존재합니다', c)
    print('------------------------------------------------------------------------------------------------------') 
    

array(['결과 기타', '결과 기타 추가', '고객 요청', '고객 요청 사항', '교체 요청', '교체 요청 회선',
       '기타 추가', '기타 추가 연락처', '단말 특이', '단말 특이 사항', '들어옴 교체', '들어옴 교체 요청',
       '방문 지니', '방문 지니 셋탑', '사항 연락', '사항 연락 방문', '사항 진단', '사항 진단 결과',
       '셋탑 원안', '셋탑 원안 들어옴', '연락 방문', '연락 방문 지니', '요청 사항', '요청 사항 연락',
       '요청 회선', '요청 회선 단말', '원안 들어옴', '원안 들어옴 교체', '지니 셋탑', '지니 셋탑 원안',
       '진단 결과', '진단 결과 기타', '추가 연락처', '특이 사항', '특이 사항 진단', '회선 단말',
       '회선 단말 특이'], dtype=object)

In [ ]:
candidates

array(['결과 기타', '결과 기타 추가', '고객 요청', '고객 요청 사항', '교체 요청', '교체 요청 회선',
       '기타 추가', '기타 추가 연락처', '단말 특이', '단말 특이 사항', '들어옴 교체', '들어옴 교체 요청',
       '방문 지니', '방문 지니 셋탑', '사항 연락', '사항 연락 방문', '사항 진단', '사항 진단 결과',
       '셋탑 원안', '셋탑 원안 들어옴', '연락 방문', '연락 방문 지니', '요청 사항', '요청 사항 연락',
       '요청 회선', '요청 회선 단말', '원안 들어옴', '원안 들어옴 교체', '지니 셋탑', '지니 셋탑 원안',
       '진단 결과', '진단 결과 기타', '추가 연락처', '특이 사항', '특이 사항 진단', '회선 단말',
       '회선 단말 특이'], dtype=object)

In [ ]:
candidates

['단말 특이 사항', '기타 추가 연락처']


In [ ]:
top_n = 2
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

['교체 요청 회선', '들어옴 교체', '결과 기타 추가', '고객 요청', '기타 추가']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)   